In [38]:
#!pip install -q ipython-autotime
%load_ext autotime
%load_ext autoreload
%timeit
%autoreload 2

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 203 ms (started: 2023-07-25 11:51:44 +03:00)


In [39]:
import pandas as pd
import numpy as np
import torch 
from torch import nn
from Utils import *
from preprocess import *
from models import ANN, CNN1D, BILSTM
from torch.utils.data import Dataset, DataLoader



time: 188 ms (started: 2023-07-25 11:51:44 +03:00)


In [40]:
loaded_data = np.load("cleaned-dataset\data_npy.npy")
X_train, X_test, y_train, y_test = split_data(loaded_data, ratio=0.70, seed=42)

training_dataset = TextDataset(X_train, y_train)
testing_dataset = TextDataset(X_test, y_test)
train_loader = DataLoader(dataset=training_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=training_dataset, batch_size=128, shuffle=True)

time: 203 ms (started: 2023-07-25 11:51:44 +03:00)


In [41]:
def acc_func(y_true, y_pred):
    y_pred = torch.round(y_pred)
    return (y_true == y_pred).sum() / len(y_true)

time: 250 ms (started: 2023-07-25 11:51:45 +03:00)


In [42]:
x, y = next(iter(train_loader))
x.shape, x.unsqueeze(1).shape, x.unsqueeze(-1).shape, x.unsqueeze(0).shape

(torch.Size([128, 100]),
 torch.Size([128, 1, 100]),
 torch.Size([128, 100, 1]),
 torch.Size([1, 128, 100]))

time: 187 ms (started: 2023-07-25 11:51:45 +03:00)


In [43]:
#* GLOABL VARIABLES
EPOCHS = 150
LEARNING_RATE = 3e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("cpu")

#* LSTM 
INPUT_SIZE = train_loader.dataset[0][0].shape[0]
HIDDEN_STATE = 64
NUM_LAYERS = 4
NUM_CLASSES = 1 #* binary classification

model = ANN().to(DEVICE)
# model = BILSTM(INPUT_SIZE, HIDDEN_STATE, NUM_LAYERS, NUM_CLASSES, bidirection=True).to(DEVICE)


loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)





time: 328 ms (started: 2023-07-25 11:51:45 +03:00)


In [44]:
# torch.save(ann.state_dict(), "model.pth")
# ann_loaded = ANN().to(DEVICE)
# ann_loaded.load_state_dict(torch.load("model.pth"))

time: 281 ms (started: 2023-07-25 11:51:45 +03:00)


In [45]:
history = Training(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    DEVICE,
    loss,
    optimizer,
    print_every=1,
    load_saved_model=True,
)

Loaded saved model ANN_best_model.pth successfully
Epoch 0 | Train Loss: 0.06682
Epoch 0 | Train Acc: 97.34%
Epoch 0 | Test Loss: 0.03679
Epoch 0 | Test Acc: 98.85%
Test loss decreased to 0.03679 saving new best model
--------------------------------------------------
Epoch 1 | Train Loss: 0.06642
Epoch 1 | Train Acc: 97.45%
Epoch 1 | Test Loss: 0.03669
Epoch 1 | Test Acc: 98.60%
Test loss decreased from 0.03679 to 0.03669 saving new best model
--------------------------------------------------
Interrupted, returning saved history
time: 7.42 s (started: 2023-07-25 11:51:46 +03:00)


In [ ]:
# loaded_ann = ANN().to(DEVICE)
# loaded_ann.load_state_dict(torch.load("ANN_best_model.pth"))

In [46]:
path = "C:\\Users\\DELL\\OneDrive\\Desktop\\AI projects\\Fake-News-Detection\\word2vec_model\\word2vec.model"
word2vec_model = Load_word2vec(path)




Word2Vec model loaded successfully!
time: 1.59 s (started: 2023-07-25 11:51:56 +03:00)


In [49]:
from preprocess import *
sample = "The billionaire changed his profile picture to the new logo and added X.com to his Twitter bio."

predict(sample, word2vec_model, model, DEVICE, prepare_for=model._get_name())
# sample = preprocess_text(sample)
# sample = vectorize_text(sample, word2vec_model)
# sample = torch.tensor(sample).float().to(DEVICE).unsqueeze(0).unsqueeze(1)
# print(sample.shape)
# model.eval()
# model(sample)
# print(predict(sample, word2vec_model, loaded_ann, DEVICE))

'Real, with probability: 73.26%'

time: 250 ms (started: 2023-07-25 11:52:33 +03:00)


In [ ]:
# Importing metrics
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Prediction
y_pred = ann(torch.tensor(X_test).float().to(DEVICE)).to("cpu").detach().numpy().reshape(-1)


# Accuracy score
print("Model accuracy is ",accuracy_score(y_pred,y_test))

confusion_matrix = confusion_matrix(y_pred=y_pred,y_true=y_test)

fig,ax = plt.subplots(figsize=(6,6))
sns.heatmap(confusion_matrix,annot=True,fmt="0.1f",linewidths=1.5)
plt.show()